In [ ]:
# Single-cell aria2 wrapper
import os, sys, time, datetime, subprocess, shutil, glob, re
from google.colab import drive

# ---------- ensure aria2 present ----------
from shutil import which
if not which("aria2c"):
    print("aria2c not found — installing aria2 (only runs if missing)...")
    subprocess.run(["apt-get","update","-qq"], check=True)
    subprocess.run(["apt-get","install","-qq","-y","aria2"], check=True)

# ---------- mount drive ----------
drive.mount('/content/drive', force_remount=False)
DRIVE_BASE = "/content/drive/MyDrive/Torrent"
os.makedirs(DRIVE_BASE, exist_ok=True)

# ---------- cleanup old temp folders created by this script ----------
for p in sorted(glob.glob("/content/temp_torrent_download*")):
    try:
        if os.path.isdir(p):
            shutil.rmtree(p)
        else:
            os.remove(p)
    except Exception:
        pass

# ---------- user options ----------
zip_choice = input("Zip after download? (Y/y to zip, else leave blank): ").strip()
want_zip = zip_choice.lower() in ("y","yes")
raw_zip_name = None
if want_zip:
    raw_zip_name = input("Enter zip filename (with or without .zip). Leave blank for timestamped name: ").strip()
    if not raw_zip_name:
        raw_zip_name = "torrent_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_name_no_ext = os.path.splitext(raw_zip_name)[0]
    final_zip_path = os.path.join(DRIVE_BASE, zip_name_no_ext + ".zip")
    # use temp local folder for download when zipping
    download_dir = "/content/temp_torrent_download_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    os.makedirs(download_dir, exist_ok=True)
    folder_name = None
else:
    # user provides a folder name (used if we need to create a destination when moving multiple items)
    folder_name = input("Enter folder name inside Torrent (leave blank for timestamped): ").strip()
    if not folder_name:
        folder_name = "torrent_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # For convenience we still download into a temp folder, but we will move only the content (not the temp root)
    # This keeps behavior consistent and avoids partial-write issues on Drive during download.
    download_dir = "/content/temp_torrent_download_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    os.makedirs(download_dir, exist_ok=True)
    final_zip_path = None

print(f"\n📁 Download directory: {download_dir}")
if final_zip_path:
    print(f"📦 Final zip path: {final_zip_path}")

# ---------- get magnet or .torrent ----------
print("\nPaste a magnet URI, paste a full path to a .torrent file, or press Enter to pick a .torrent from Drive/Torrent.")
user_in = input("Magnet or .torrent path (or Enter to browse): ").strip()

is_magnet = False
torrent_path = None
if user_in:
    if user_in.lower().startswith("magnet:"):
        is_magnet = True
        uri = user_in
    elif user_in.lower().endswith(".torrent") and os.path.exists(user_in):
        torrent_path = user_in
    else:
        candidate = os.path.join(DRIVE_BASE, user_in)
        if user_in.lower().endswith(".torrent") and os.path.exists(candidate):
            torrent_path = candidate
        else:
            raise SystemExit("Input not recognized as magnet or existing .torrent. Aborting.")
else:
    found = sorted(glob.glob(os.path.join(DRIVE_BASE, "*.torrent")))
    if not found:
        raise SystemExit("No .torrent files found in Drive/Torrent. Upload one or paste a magnet and re-run.")
    print("\nFound .torrent files in Drive/Torrent:")
    for i,f in enumerate(found, start=1):
        print(f" {i}. {os.path.basename(f)}")
    sel = input("Enter number to use that .torrent: ").strip()
    if not sel.isdigit():
        raise SystemExit("Invalid selection.")
    torrent_path = found[int(sel)-1]

# ---------- build aria2 command ----------
aria2_log = "/tmp/aria2_stream.log"
if os.path.exists(aria2_log):
    try: os.remove(aria2_log)
    except: pass

aria2_cmd = [
    "aria2c",
    "-x", "16",                    # connections per server
    "--dir", download_dir,
    "--file-allocation=none",
    "--seed-time=0",             # exit after finished
    "--summary-interval=1",
    "--enable-dht=true",
    "--enable-peer-exchange=true",
    "--bt-enable-lpd=true",
    "--log=" + aria2_log,
    "--log-level=info",
]

if is_magnet:
    aria2_cmd += [uri]
else:
    aria2_cmd += [torrent_path]

# ---------- run aria2c and stream parsed, friendly output ----------
print("\n🔍 Stage: Downloading metadata (aria2 running)...")
proc = subprocess.Popen(aria2_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)

saw_metadata = False
saw_content_progress = False
last_print_time = 0

# regexes
prog_re = re.compile(r'([\d\.]+[KMGTP]?i?B)\s*/\s*([\d\.]+[KMGTP]?i?B)\s*\((\d+)%\).*DL: *([\d\.]+[KMGTP]?i?B)', re.IGNORECASE)
metadata_re = re.compile(r'METADATA', re.IGNORECASE)

try:
    for raw_line in proc.stdout:
        line = raw_line.strip()
        # metadata detection
        if not saw_metadata and metadata_re.search(line):
            saw_metadata = True
            print("✅ Stage: Metadata downloaded.")
            print("⬇️  Stage: Starting content download (may take time to find peers)...")
            continue

        # progress detection
        m = prog_re.search(line)
        if m:
            done_txt, total_txt, pct_txt, speed_txt = m.group(1), m.group(2), m.group(3), m.group(4)
            now = time.time()
            if now - last_print_time > 5:
                print(f"🔄 Downloading — {pct_txt}%  {done_txt}/{total_txt}  |  {speed_txt}/s")
                last_print_time = now
            saw_content_progress = True
            continue

        # detect completion lines
        if ("download complete" in line.lower() or "download finished" in line.lower()) and not line.lower().count("metadata"):
            print("✅ Stage: Download finished (aria2 reported completion).")
            break

        # detect seeding lines as post-complete signal
        if saw_content_progress and line.upper().startswith("[#") and "SEED" in line.upper():
            if time.time() - last_print_time > 1:
                print("✅ Stage: Content reached seeding state (treating as finished).")
                break

    # finalize
    proc.wait(timeout=10)

except Exception as e:
    try:
        proc.terminate()
    except Exception:
        pass
    raise

# ---------- check aria2 exit code ----------
rc = proc.returncode
if rc != 0:
    print("\n⚠️ aria2 exited with code", rc)
    try:
        with open(aria2_log, "r", encoding="utf-8", errors="replace") as f:
            tail = f.read()[-4000:]
            print("aria2 log tail:\n", tail)
    except Exception as e:
        print("Could not read aria2 log:", e)
    raise SystemExit("aria2 error — see log above.")

# ---------- remove leftover aria2 control files ----------
removed = []
for pat in ("**/*.aria2","**/*.aria2part","**/*.aria2c","**/*.aria2_control"):
    for p in glob.glob(os.path.join(download_dir, pat), recursive=True):
        try:
            os.remove(p); removed.append(p)
        except Exception:
            pass
if removed:
    print(f"🧹 Removed {len(removed)} aria2 control file(s).")

# ---------- show top-level items ----------
top_items = sorted([n for n in os.listdir(download_dir) if not n.startswith('.')])
print("\n📄 Top-level items in download directory:")
for name in top_items[:100]:
    print(" -", name)

# ---------- helper: unique dest name ----------
def unique_path(path):
    if not os.path.exists(path):
        return path
    base, ext = os.path.splitext(path)
    i = 1
    while True:
        cand = f"{base}_{i}{ext}"
        if not os.path.exists(cand):
            return cand
        i += 1

# ---------- zip or move logic ----------
if want_zip:
    print("\n📦 Zipping files, please wait...")
    base_no_ext = os.path.splitext(final_zip_path)[0]
    shutil.make_archive(base_no_ext, 'zip', download_dir)
    print(f"🎉 Zipping complete. Saved: {final_zip_path}")
    try:
        shutil.rmtree(download_dir)
        print("🧹 Removed temporary download folder.")
    except Exception:
        pass
else:
    # Smart move: if the temp directory holds exactly one top-level directory, move that dir only.
    if download_dir.startswith("/content/temp_torrent_download"):
        if len(top_items) == 1 and os.path.isdir(os.path.join(download_dir, top_items[0])):
            child_path = os.path.join(download_dir, top_items[0])
            dest = os.path.join(DRIVE_BASE, os.path.basename(child_path))
            dest = unique_path(dest)
            print(f"\n📂 Moving single torrent folder into Drive: {dest}")
            shutil.move(child_path, dest)
            # remove temp root
            try: shutil.rmtree(download_dir)
            except Exception: pass
            print("✅ Move complete.")
        else:
            # multiple items -> create a folder and move items into it
            dest_folder = os.path.join(DRIVE_BASE, folder_name)
            dest_folder = unique_path(dest_folder)
            os.makedirs(dest_folder, exist_ok=True)
            print(f"\n📂 Moving all top-level items into Drive folder: {dest_folder}")
            for item in top_items:
                src = os.path.join(download_dir, item)
                try:
                    shutil.move(src, dest_folder)
                except Exception:
                    # fallback: try copy then remove
                    if os.path.isdir(src):
                        shutil.copytree(src, os.path.join(dest_folder, item))
                        shutil.rmtree(src, ignore_errors=True)
                    else:
                        shutil.copy2(src, os.path.join(dest_folder, item))
                        os.remove(src)
            try: shutil.rmtree(download_dir)
            except Exception: pass
            print("✅ Move complete.")
    else:
        # downloaded directly to Drive (shouldn't happen with current logic) — nothing to move
        print("\n📂 Download was saved directly to Drive at:", download_dir)

print("\n✅ ALL DONE.")
